In [1]:
import warnings
warnings.filterwarnings('ignore')

### Ý tưởng của Thuật toán cosine-similarity:
- Số hóa dữ liệu của bảng
- Đưa bẳng về dạng vector chứa các trường tương ứng với hàng/cột
- Đo độ tương đồng của vector tham chiếu đến bài hát với các vector khác, công thức theo hình
- Xếp độ tương đồng cao đến thấp => bài nào có độ chênh thấp tức đặc điểm dữ liệu giống => là bài nên recommend

In [2]:
import pandas as pd
import json
import numpy as np

Đọc dữ liệu

In [3]:
df = pd.read_csv('song_data-test.csv')
print(df.shape)
df.tail()

(391969, 21)


,title,artist,area,youtube_link,spotify_link,youtube_view_count,youtube_like_count,acousticness,danceability,duration_ms,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
391964,Come Back Home,Calum Scott,NaN,NaN,0cvfSKcm9VeduwyYPrxtLx,NaN,NaN,0.00678,0.601,190539.0,...,0.0,11.0,0.0991,-5.174,1.0,0.0323,131.049,4.0,0.2890,57.0
391965,Enough for You,Saint Claire,NaN,NaN,43MP9F7UzvfilSrw2SqZGJ,NaN,NaN,0.91800,0.387,194583.0,...,0.0,9.0,0.1030,-13.233,1.0,0.0437,94.039,4.0,0.3460,60.0
391966,Do It,Mike Stud,NaN,NaN,4TWlUuFk81NGUNKwndyS5Q,NaN,NaN,0.33000,0.717,139191.0,...,0.0,8.0,0.0997,-8.351,0.0,0.2060,156.977,4.0,0.5460,47.0
391967,No Smoke,D Savage,NaN,NaN,5iGBXzOoRo4sBTy8wdzMyK,NaN,NaN,0.00790,0.772,180013.0,...,0.0,4.0,0.1310,-9.670,0.0,0.1200,120.049,4.0,0.0755,50.0
391968,En Vida,Banda Los Sebastianes,NaN,NaN,7LNtyuekYHiZ99UxkrfCQR,NaN,NaN,0.54900,0.715,145827.0,...,0.0,3.0,0.1080,-3.244,1.0,0.0367,130.128,3.0,0.9760,55.0


In [4]:
df['artist']

0                      "#Botsu"
1         "El Cholo" Valderrama
2         "El Cholo" Valderrama
3         "El Cholo" Valderrama
4         "El Cholo" Valderrama
                  ...          
391964              Calum Scott
391965             Saint Claire
391966                Mike Stud
391967                 D Savage
391968    Banda Los Sebastianes
Name: artist, Length: 391969, dtype: object

In [5]:
import random
seed_value = 42
data = json.load(open('playlist.json'))
dff = df[df['spotify_link'].isin(set([x for xs in data for x in xs]))]

Columns: Index(['title', 'artist', 'area', 'youtube_link', 'spotify_link',
       'youtube_view_count', 'youtube_like_count', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'popularity'],
      dtype='object')


**Lưu lại một bản dữ liệu để làm đầu vào trước khi xử lý**

In [6]:
dfff = dff

In [7]:
#dff.drop(['duration_ms','youtube_link','spotify_link'],axis=1,inplace=True)

In [8]:
dff = dff[['artist','youtube_view_count','acousticness','danceability','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence','popularity']]

In [9]:
import ast

def fetch_list(strings):
    try:
        result = ast.literal_eval(strings)
        return result if isinstance(result, list) else [result]
    except (SyntaxError, ValueError):
        try:
            return [s.strip('"\'') for s in strings.split(',')]
        except Exception as e:
            print(f"Error processing expression in {strings}. Error: {e}")
            return []

# Print the 'artist' column before applying the fetch_list function
print("Before:")
print(dff['artist'])

# Apply the modified fetch_list function to the 'artist' column
dff['artist'] = dff['artist'].apply(fetch_list)

# Print the 'artist' column after applying the fetch_list function
print("\nAfter:")
print(dff['artist'])


Before:
0              "#Botsu"
20         'Til Tuesday
21         'Til Tuesday
22         'Til Tuesday
63             (G)I-DLE
              ...      
300868        Deebo Mac
300869    Monster Rally
300871    DirtyPaperMan
300876        Dj Lindão
300878           Siggno
Name: artist, Length: 34443, dtype: object

After:
0                [#Botsu]
20          [Til Tuesday]
21          [Til Tuesday]
22          [Til Tuesday]
63             [(G)I-DLE]
               ...       
300868        [Deebo Mac]
300869    [Monster Rally]
300871    [DirtyPaperMan]
300876        [Dj Lindão]
300878           [Siggno]
Name: artist, Length: 34443, dtype: object


**Mã hóa các dữ liệu không phải kiểu dạng numberic**


In [10]:
dff['artist'] = dff['artist'].apply(lambda x: " ".join(map(str, x)))

In [11]:
dff['artist'] = dff['artist'].apply(lambda x: x.lower())

In [12]:
dff['artist'].value_counts()

artist
the beatles       367
taylor swift      319
stray kids        204
justin bieber     200
the weeknd        196
                 ... 
jonny diaz          1
artifice            1
harrison brome      1
laura misch         1
dj lindão           1
Name: count, Length: 4433, dtype: int64

In [13]:
dff['artist'].unique()

array(['#botsu', 'til tuesday', '(g)i-dle', ..., 'monster rally',
       'dirtypaperman', 'dj lindão'], dtype=object)

In [14]:
dff['artist']=pd.factorize(dff['artist'])[0]

In [ ]:
**Dữ liệu sau khi mã hóa**

In [16]:
dff['mode'] = dff['mode'].astype(bool)
dff=dff.fillna(0)
dff

,artist,youtube_view_count,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0,4811528.0,0.0157,0.518,0.982,0.989000,4.0,0.0328,-7.130,False,0.0434,136.033,4.0,0.498,0.0
20,1,2528148.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
21,1,2018645.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
22,1,2528152.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
63,2,287408093.0,0.3680,0.583,0.576,0.006520,11.0,0.0587,-14.453,True,0.0295,108.030,4.0,0.881,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300868,4429,2999218.0,0.4180,0.683,0.776,0.000000,1.0,0.0599,-5.352,False,0.1280,98.018,4.0,0.434,23.0
300869,4430,858048.0,0.3830,0.590,0.432,0.904000,8.0,0.0805,-8.660,True,0.2100,132.072,4.0,0.454,32.0
300871,4431,1079440.0,0.0131,0.580,0.582,0.003140,8.0,0.5480,-8.036,False,0.0391,130.041,4.0,0.395,0.0
300876,4432,960401.0,0.1380,0.947,0.334,0.000000,1.0,0.1660,-5.917,True,0.2110,130.000,4.0,0.957,55.0


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
vectors = dff.to_numpy()

**Chuẩn hóa vector**

In [19]:
vectors

array([[0, 4811528.0, 0.0157, ..., 4.0, 0.498, 0.0],
       [1, 2528148.0, 0.0596, ..., 4.0, 0.618, 0.0],
       [1, 2018645.0, 0.0596, ..., 4.0, 0.618, 0.0],
       ...,
       [4431, 1079440.0, 0.0131, ..., 4.0, 0.395, 0.0],
       [4432, 960401.0, 0.138, ..., 4.0, 0.957, 55.0],
       [4189, 960401.0, 0.258, ..., 4.0, 0.703, 19.0]], dtype=object)

Tính cosine_similarity (VD ma trận trên):

![image.png](../../resource/img/CS.png)


Kết quả thu được là mảng 2 chứa độ tương thích của các bài hát (các phần tử trong vector)

In [20]:
similarity = cosine_similarity(vectors)

In [6]:
#sorted(similarity[0],reverse=True)
#list(enumerate(similarity[0]))

In [24]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(24, 0.9999999999989835),
 (71, 0.9999999999985743),
 (72, 0.9999999999985743),
 (73, 0.9999999999985743),
 (22, 0.9999999999983611),
 (55, 0.9999999999979011),
 (28, 0.9999999999975966),
 (34, 0.9999999999975542),
 (61, 0.9999999999972368),
 (26, 0.9999999999968247)]

In [25]:
dfff

,title,artist,area,youtube_link,spotify_link,youtube_view_count,youtube_like_count,acousticness,danceability,duration_ms,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,(darude) sandstorm (original mix)[www homeofmu...,"""#Botsu""",JP,https://www.youtube.com/watch?v=61-PXrbs4MA,2lylyZl9S7rbp2FUP5IS0r,4811528.0,44569.0,0.0157,0.518,229573.0,...,0.989000,4.0,0.0328,-7.130,0.0,0.0434,136.033,4.0,0.498,NaN
20,Voices Carry,'Til Tuesday,USUK,https://www.youtube.com/watch?v=9pv1UVA1M0Y,04pLdXwdbFUSezL2Gtjp5r,2528148.0,34387.0,0.0596,0.598,260400.0,...,0.000001,7.0,0.1700,-11.537,1.0,0.0245,105.228,4.0,0.618,NaN
21,Coming Up Close,'Til Tuesday,USUK,https://www.youtube.com/watch?v=wtOgwFzhlyw,5jGu0X5t3jTS40qDGqJ0Kv,2018645.0,13912.0,0.0596,0.598,260400.0,...,0.000001,7.0,0.1700,-11.537,1.0,0.0245,105.228,4.0,0.618,NaN
22,What About Love,'Til Tuesday,USUK,https://www.youtube.com/watch?v=DZ48dMnqAy4,03YERsfj67SCf84wzy7w1a,2528152.0,34388.0,0.0596,0.598,260400.0,...,0.000001,7.0,0.1700,-11.537,1.0,0.0245,105.228,4.0,0.618,NaN
63,TOMBOY,(G)I-DLE,KR,https://www.youtube.com/watch?v=Jh4QFaPmdss,0IGUXY4JbK18bu9oD4mPIm,287408093.0,3612708.0,0.3680,0.583,218373.0,...,0.006520,11.0,0.0587,-14.453,1.0,0.0295,108.030,4.0,0.881,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300868,My Grind,Deebo Mac,JP,https://www.youtube.com/watch?v=YPm9T622OX0,3AmRW8aTemk24nNfrZz4C7,2999218.0,83131.0,0.4180,0.683,240039.0,...,0.000000,1.0,0.0599,-5.352,0.0,0.1280,98.018,4.0,0.434,23.0
300869,Flight to the Jungle (Landing),Monster Rally,JP,https://www.youtube.com/watch?v=cr9iLdXK9ik,0O8lPdpJDM2u9OYFxwnW4Q,858048.0,39594.0,0.3830,0.590,296117.0,...,0.904000,8.0,0.0805,-8.660,1.0,0.2100,132.072,4.0,0.454,32.0
300871,Wrist Froze,DirtyPaperMan,JP,https://www.youtube.com/watch?v=eZWjYWcQ4Xo,4zk3kiCivArCGqf5XN8Vqs,1079440.0,32772.0,0.0131,0.580,96026.0,...,0.003140,8.0,0.5480,-8.036,0.0,0.0391,130.041,4.0,0.395,0.0
300876,Vai Faz a Fila 2,Dj Lindão,JP,https://www.youtube.com/watch?v=j69l0KUkltw,5LFGLDN6zIxJqDeuZU1Ncx,960401.0,8588.0,0.1380,0.947,182831.0,...,0.000000,1.0,0.1660,-5.917,1.0,0.2110,130.000,4.0,0.957,55.0


### Giải thích recommend_common_song:
 Input: 
    - [spotiy_link]: Mảng id của bài hát
    - songs_list: Ma trận của songs
- Output:
    - Mảng id các bài hát recommend từ các bài cho trước


In [35]:
#dfff[dfff["name"]=='Sister'].index
def recommend(song):
    song_index = dfff[dfff["title"]==song].index[0]
    distance = similarity[song_index]
    songs_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:11]
    for i in songs_list:
        print(dfff.iloc[i[0]]['title'])

In [36]:
def recommend_common_songs_multi(*songs):
    # Tạo một từ điển để lưu trữ tổng điểm của mỗi bài hát
    song_scores = {}
    
    # Duyệt qua từng bài hát
    for song in songs:
        # Lấy chỉ số của bài hát trong DataFrame dfff
        song_index = dfff[dfff["spotify_link"] == song].index[0]
        
        # Lấy vectơ tương tự của bài hát đầu vào
        distance = similarity[song_index]
        
        # Sắp xếp các bài hát dựa trên độ tương tự và lấy 10 bài hát tương tự nhất
        songs_list = sorted(list(enumerate(distance)), reverse=True)
        
        # Tính điểm cho mỗi bài hát trong song_list và cộng tổng điểm vào từ điển
        for i, (_, similarity_score) in enumerate(songs_list):
            song_name = dfff.iloc[i]['spotify_link']
            song_scores[song_name] = song_scores.get(song_name, 0) + (i + 1) * similarity_score
    
    # Lấy 10 bài hát có tổng điểm cộng thấp nhất
    common_songs = sorted(song_scores.items(), key=lambda x: x[1])[:10]
    
    # In ra tên của 10 bài hát có tổng điểm cộng thấp nhất
    for song, score in common_songs:
        print(f"{song}: {score}")

### Ví dụ minh họa

In [37]:
recommend_common_songs_multi("2lylyZl9S7rbp2FUP5IS0r", "04pLdXwdbFUSezL2Gtjp5r")

2lylyZl9S7rbp2FUP5IS0r: 1.9999809566812312
04pLdXwdbFUSezL2Gtjp5r: 3.999957377457492
5jGu0X5t3jTS40qDGqJ0Kv: 5.999949421513667
03YERsfj67SCf84wzy7w1a: 7.999893306510083
0IGUXY4JbK18bu9oD4mPIm: 9.999989096630884
4uOBL4DDWWVx4RhYKlPbPC: 11.999984685870466
6NnCWIWV740gP7DQ8kqdIE: 13.999913020935697
2DmRXiyn03tOqKgEJXlaiJ: 15.999942362906886
1t8sqIScEIP0B4bQzBuI2P: 17.99999969333105
2ezKXygNO30pXyDQXkm6oD: 19.999809796730638


In [29]:
recommend('What About Love')

LIAR
Change
Kill this
sober
city flow
Creepy Nightmare
(darude) sandstorm (original mix)[www homeofmusic com]
155
HWAA - English Version
Oh my god - Japanese ver.


In [30]:
dff

,artist,youtube_view_count,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0,4811528.0,0.0157,0.518,0.982,0.989000,4.0,0.0328,-7.130,False,0.0434,136.033,4.0,0.498,0.0
20,1,2528148.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
21,1,2018645.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
22,1,2528152.0,0.0596,0.598,0.515,0.000001,7.0,0.1700,-11.537,True,0.0245,105.228,4.0,0.618,0.0
63,2,287408093.0,0.3680,0.583,0.576,0.006520,11.0,0.0587,-14.453,True,0.0295,108.030,4.0,0.881,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300868,4429,2999218.0,0.4180,0.683,0.776,0.000000,1.0,0.0599,-5.352,False,0.1280,98.018,4.0,0.434,23.0
300869,4430,858048.0,0.3830,0.590,0.432,0.904000,8.0,0.0805,-8.660,True,0.2100,132.072,4.0,0.454,32.0
300871,4431,1079440.0,0.0131,0.580,0.582,0.003140,8.0,0.5480,-8.036,False,0.0391,130.041,4.0,0.395,0.0
300876,4432,960401.0,0.1380,0.947,0.334,0.000000,1.0,0.1660,-5.917,True,0.2110,130.000,4.0,0.957,55.0


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'dff' is your original DataFrame
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

# Sample a subset of data
dff_sampled = dff.sample(n=10000).reset_index(drop=True)

# Split the data into training and testing sets
train_data, test_data = train_test_split(dff_sampled, test_size=0.2, random_state=seed_value)

# Fill NaN values with 0
train_data_filled = train_data.fillna(0)

# Convert the training data to a numpy array
vectors_train = train_data_filled.to_numpy()

# Compute cosine similarity
similarity_train = cosine_similarity(vectors_train)

# Display the first few rows of the similarity matrix
print(similarity_train[:5, :5])


[[1.         0.99999956 0.99999971 0.9999986  1.        ]
 [0.99999956 1.         0.99999998 0.99999659 0.99999947]
 [0.99999971 0.99999998 1.         0.99999705 0.99999964]
 [0.9999986  0.99999659 0.99999705 1.         0.99999875]
 [1.         0.99999947 0.99999964 0.99999875 1.        ]]


In [32]:
import numpy as np

def get_items_rated_by_user(rates, user_idx):
    """
    Get items (indices) rated by a specific user.

    Parameters:
    - rates (numpy.ndarray): User-item rating matrix.
    - user_idx (int): Index of the user.

    Returns:
    - Tuple: Indices and scores of items rated by the user.
    """
    items_rated = np.where(rates[:, user_idx] > 0)[0]
    
    if len(items_rated) == 0:
        return [], []
    
    scores = rates[items_rated, user_idx]
    return items_rated, scores

def calculate_rmse(Yhat, rates):
    se = 0
    cnt = 0
    for n in range(min(len(Yhat[0]), rates.shape[1])):
        if np.any(rates[:, n] > 0):  # Check if the user has rated any items
            ids, scores_truth = get_items_rated_by_user(rates, n)
            scores_pred = Yhat[ids, n]
            e = scores_truth - scores_pred 
            se += (e * e).sum(axis=0)
            cnt += e.size
    return np.sqrt(se / cnt)

print("Before calculating RMSE:")
print("similarity_train shape:", similarity_train.shape)
print("vectors_train shape:", vectors_train.shape)

rmse_train = calculate_rmse(similarity_train, vectors_train)
print(f'RMSE for training: {rmse_train}')


Before calculating RMSE:
similarity_train shape: (8000, 8000)
vectors_train shape: (8000, 15)
RMSE for training: 44803976.069832996


In [33]:
data


[['1c6C2wg7PRbPBRRUaEvheC',
  '5qba0ZfIK6Y3n2ZNuyhYqC',
  '6YZ5KxfrGopg7r3aqjKio7',
  '1UTAMofVtl1IcRpXFRwnRR',
  '6wHKRTOzeuQ3wnteyyx6gc',
  '00ViGJj0kfI366ZcWdpgmr',
  '08f0fHqZigPgPHCfRjk8SC',
  '3fTU9APFHQbJrptArAR0yu',
  '2K25JuNZtuPJ4PP8TC07BT',
  '7KvcQ2s6TFfL9GjklLFy24',
  '2Xvjhf7ZECR2hwaCAcsFgb',
  '3nP0NLaVnAQNWOo2R6ouyY',
  '0KC3pA3FuHWEeHdhLVHHVs',
  '0bMG9mHnuAxsb18b2SLAVD',
  '0Sayb1EWTywPttwJo7zjBt',
  '6JZ59QeUpGaA1zP5xP2kSU',
  '7CWT9FxUOzFBPI7sUKjada',
  '6IWpcw8iWD8j5sNAbekRqq',
  '1ufaNhO1DPlSiynfO2OtIj',
  '53Tx9xMaE5U8Gv6h9nYptl',
  '009G1RDIr3UgPrFzOPJPfb',
  '4ZkhFcoS3apzze9w2yI9NO',
  '24SkzbH7hGhUvqIaKlJ1TY',
  '3gSR5f8bV4GyOKyO53lAkK',
  '3gqhQruuoUgcijvq4cSXmZ',
  '1W0OCrMsRP1SLTEdw3JwEz',
  '4mLpAV6DXOdC8a2dlIxRSM',
  '0dFhDyKsIyupKGndzqppVM',
  '5dB0iQURzcObhAsmmp568o',
  '06sLwX6s6BhtD4NjmP9F40',
  '2PFcsKaB1kweBErBYZiPB0',
  '72Qo3U5apeuv91j9BEqJBM',
  '4oTzuu10mN7yQSbYk8m56o',
  '4Li2WHPkuyCdtmokzW2007',
  '2evx7AGgniSwmrszZOh60Q',
  '5eXdPx9KDmozFmfJm

In [34]:
file_list = ["playlist.json"]